# Importer les bibliothèques nécessaires

In [2]:
import pandas as pd
import ast 
import numpy as np
import csv

# Importer les données de Doctolib

In [3]:
data = pd.read_csv('data/extracted_data_final_0_1000.csv')

In [4]:
# Fonction pour réorganiser les profils cards
def reorder_cards(profile_cards):
    try:
        cards_dict = ast.literal_eval(profile_cards)
        
        reordered_cards = {
            'card_0': np.nan,
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

        def split_content(value):
            value = value.replace('\\n', '\n').replace('\\', '\n')
            return value.split('\n')

        for key, value in cards_dict.items():
            #print(key,value,'s')
            if value.startswith("Carte et informations d'accès"):
                #print('1')
                reordered_cards['card_1'] = split_content(value)
            elif value.startswith("Présentation"):
                #print('2')
                reordered_cards['card_2'] = split_content(value)
            elif value.startswith("Horaires et coordonnées"):
                #print('3')
                reordered_cards['card_3'] = split_content(value)
            elif value.startswith("Tarifs et remboursement") or value.startswith("Lieu"):
                #print('i')
                reordered_cards['card_0'] = split_content(value)
            elif value.startswith("Tarifs"):
                #print('4')
                reordered_cards['card_4'] = split_content(value)
            elif value.startswith("Informations légales"):
                #print('5')
                reordered_cards['card_5'] = split_content(value)
            else :
                if isinstance(value,list):
                    #print('i1')
                    reordered_cards['card_0'] = split_content(value)
        
        return reordered_cards
    except:
        return {
            'card_0': np.nan,
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

reordered_cards = data['Profile Cards'].apply(reorder_cards)

reordered_cards_df = pd.json_normalize(reordered_cards)

data_combined = pd.concat([data, reordered_cards_df], axis=1)

data_combined.drop(columns = 'Profile Cards', inplace=True)
print(data_combined[['Doctor Name', 'card_0', 'card_1', 'card_2', 'card_3', 'card_4', 'card_5']].tail(9))


                   Doctor Name  \
991        Dr Mathieu SABATIER   
992           Dr Eric LE FLOCH   
993    Agnès Montagnon -Thomas   
994     Dr Stella Danan-Hadida   
995  Dr Philippe Lazare SELLAM   
996          Dr Michèle Maizil   
997           Dr Thierry MEYER   
998          Dr Marc Stromboni   
999               Fabien GUIDE   

                                                card_0  \
991                                                NaN   
992  [Lieu 1, Lieu 2, Lieu 3, Radio Faidherbe, Radi...   
993  [Tarifs et remboursement, , , Tiers payant : S...   
994  [Tarifs et remboursement, , , , Conventionné s...   
995  [Lieu 1, Lieu 2, Cabinet du Docteur Philippe L...   
996                                                NaN   
997  [Tarifs et remboursement, , , , Conventionné, ...   
998  [Tarifs et remboursement, , Conventionné secte...   
999  [Tarifs et remboursement, , , , Conventionné, ...   

                                                card_1  \
991  [Carte et infor

In [5]:
#Pour rendre la localisation plus claire
def extract_lat_lng(map_location):
    try:
        location_dict = ast.literal_eval(map_location)
        return (location_dict['lat'], location_dict['lng'])
    except:
        return (None, None)

data_combined['Latitude'], data_combined['Longitude'] = zip(*data_combined['Map Location'].apply(extract_lat_lng))

data_combined.drop(columns = 'Map Location', inplace=True)


In [6]:
# Fonction pour traiter la colonne 'Skills'
def process_skills(skills):
    if skills == 'Skills missing':
        return np.nan
    else:
        return skills.split('\n')

data_combined['Skills Processed'] = data_combined['Skills'].apply(process_skills)

data_combined.drop(columns = 'Skills', inplace=True)

In [7]:
data_combined['Specialty'].value_counts()

Specialty
Chirurgien-dentiste                                                 152
Médecin généraliste                                                 104
Ophtalmologue                                                        75
Cabinet médical                                                      56
Spécialiste en chirurgie plastique reconstructrice et esthétique     40
                                                                   ... 
Chirurgien oral                                                       1
Acupuncteur                                                           1
Addictologue                                                          1
Chirurgien cancérologue                                               1
Diététicien                                                           1
Name: count, Length: 78, dtype: int64

In [8]:
data_combined['RPPS'] = data_combined['card_5'].apply(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else np.nan)
data_combined.drop(columns = 'card_5', inplace = True)

In [9]:
data_combined['Nb Skills'] = data_combined['Skills Processed'].apply(lambda x: len(x)-1 if isinstance(x, list) else 0)

In [10]:
def paiement(card0):
    if isinstance(card0, list):
        if 'Moyens de paiement' in card0:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['moyens_paiement'] = data_combined['card_0'].apply(paiement)


def vitale(card0):
    if isinstance(card0, list):
        if 'Carte Vitale acceptée' in card0:
            return 1
        elif 'Carte Vitale non acceptée' in card0:
            return 0
        else:
            return np.nan
    else:
        return np.nan
data_combined['carte_vitale'] = data_combined['card_0'].apply(paiement)


def secteur(card0):
    if isinstance(card0, list):
        a=0
        for x in card0:
            if 'Conventionné secteur 2' in x:
                a=2
            elif 'Conventionné secteur 1' in x:
                a=1
            elif 'Conventionné secteur 3' in x:
                a=3
            elif 'Conventionné' in x:
                a=4
        return a
    else:
        return 0
data_combined['secteur'] = data_combined['card_0'].apply(secteur)

In [11]:
data_combined['Présentation']=''
data_combined['Langues parlées']=''
data_combined['Diplômes nationaux et universitaires']=''
data_combined['Autres formations']=''
data_combined['Expériences']=''
data_combined['Travaux et publications']=''
data_combined['Prix et distinctions']=''
data_combined['site']=0

liste_mots_clefs = ['Présentation','Langues parlées','Diplômes nationaux et universitaires','Formations','Autres formations','Expériences', 'Site web', 'Travaux et publications', 'Prix et distinctions']

for i in range(data_combined.shape[0]):
    
    if isinstance(data_combined['card_2'].iloc[i],list) : 
        liste_card2 = data_combined['card_2'].iloc[i][1:]
        sous_liste = ''
        label = liste_mots_clefs[0]
        for x in liste_card2 :
            if x in liste_mots_clefs :
                data_combined.loc[i, label] = sous_liste
                label = x
                sous_liste = ''
            elif x=='Voir le site':
                data_combined.loc[i, 'site'] = 1
            elif x=='▾ Voir plus':
                None
            else :
                sous_liste = sous_liste+(x)

In [12]:
data_combined['Diplômes nationaux et universitaires b']=data_combined['Diplômes nationaux et universitaires'].apply(lambda x: 1 if len(x)>0 else 0)

In [13]:
data_combined['Nb caractères présentation'] = data_combined['Présentation'].apply(lambda x: len(x))

In [14]:
data_combined['Autres formations b']=data_combined['Autres formations'].apply(lambda x: 1 if len(x)>0 else 0)

In [15]:
data_combined['Travaux et publications b']=data_combined['Travaux et publications'].apply(lambda x: 1 if len(x)>0 else 0)

In [16]:
data_combined['Nb langues']=0

for i in range(data_combined.shape[0]):
    if isinstance(data_combined['Langues parlées'].iloc[i],str) : 
        liste_langue = (data_combined['Langues parlées'].iloc[i]).split()
        count = 0
        for x in liste_langue :
            
            if  x != 'et'  :
                count +=1
        data_combined.loc[i,'Nb langues']=count

In [17]:
data_combined['Expériences b']=data_combined['Expériences'].apply(lambda x: 1 if len(x)>0 else 0)

In [18]:
data_combined['Formations b']=data_combined['Formations'].apply(lambda x: 1 if isinstance(x,str) else 0)

In [19]:
liste1 = ["Carte et informations d'accès",'Moyens de transport','Parking public','Informations pratiques',"J'autorise le traitement d'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus", 'sur la collecte et le traitement des données par Google','AFFICHER LA CARTE']
def carac1(card1):
    nb=0
    for phrase in card1:
        if  phrase not in liste1:
            nb += len(phrase)
    return nb
data_combined['Carac_card_1'] = data_combined['card_1'].apply(lambda x: carac1(x)if isinstance(x, list) else np.nan)

In [20]:
def transport(card1):
    if isinstance(card1, list):
        if 'Moyens de transport' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Transport'] = data_combined['card_1'].apply(transport)

In [21]:
def parking(card1):
    if isinstance(card1, list):
        if 'Parking public' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Parking'] = data_combined['card_1'].apply(parking)

In [22]:
def infos(card1):
    if isinstance(card1, list):
        if 'Informations pratiques' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Infos'] = data_combined['card_1'].apply(infos)

In [23]:
def coordonnées(card3):
    if isinstance(card3, list):
        if 'Coordonnées' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Coordonnées'] = data_combined['card_3'].apply(coordonnées) 

In [24]:
def sansRDV(card3):
    if isinstance(card3, list):
        if 'Consultations sans rendez-vous' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Sans RDV'] = data_combined['card_3'].apply(sansRDV)

In [25]:
data_combined.drop(columns=['URL','card_1','card_2'],inplace=True)
data_combined.head()

,Doctor Name,Specialty,card_0,card_3,card_4,Latitude,Longitude,Skills Processed,RPPS,Nb Skills,...,Travaux et publications b,Nb langues,Expériences b,Formations b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV
0,Dr Marian AGACHI,Neurochirurgien,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.878328,2.431034,"[Expertises et actes, Neurochirurgie du rachis]",10001649291,1,...,0,3,0,1,154.0,1,0,1,1,0
1,Dr Anne Vaillant Moga,Médecin généraliste,"[Lieu 1, Lieu 2, Dr Anne Moga (Paris), 76 aven...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation préalable de médecine es...",48.869218,2.285468,"[Expertises et actes, Acné, Allergie, Apnée du...",10000378991,10,...,1,2,1,0,180.0,1,0,1,0,0
2,Dr Gabriel Ohana,Chirurgien-dentiste,"[Tarifs et remboursement, , Carte Vitale accep...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation dentaire, 23 €, Traiteme...",48.894148,2.250698,"[Expertises et actes, Chirurgie buccale, Proth...",10003639373,3,...,0,1,0,0,240.0,1,0,1,1,0
3,Dr Thomas BAMBERGER,Dermatologue et vénérologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation, 31,50 € à 56,50 €, Cont...",48.815433,2.343017,NaN,10003719571,0,...,0,1,1,0,155.0,1,0,1,1,0
4,Dr Layla Chatila,Chirurgien-dentiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Coordonnées, 01 45 2...","[Tarifs, Consultation dentaire, 23 €, Ces hono...",48.858804,2.274392,NaN,10005133003,0,...,0,1,0,0,202.0,1,1,1,1,0


In [26]:
def nb_terms(card):
    if type(card)==list :
        return (len(card))
    else :
        return(0)

def nb_carac(card):
    if type(card)==list :
        s = 0
        for item in card : 
            s+=len(card)
        return(s)
    else :
        return(0)

In [27]:
data_combined['nb terms card_0']=data_combined['card_0'].apply(nb_terms)
data_combined['nb terms card_3']=data_combined['card_3'].apply(nb_terms)
data_combined['nb terms card_4']=data_combined['card_4'].apply(nb_terms)
data_combined['nb carac card_0']=data_combined['card_0'].apply(nb_carac)
data_combined['nb carac card_3']=data_combined['card_3'].apply(nb_carac)
data_combined['nb carac card_4']=data_combined['card_4'].apply(nb_carac)

# Problème des nan

In [28]:
indices_nan = data_combined[data_combined['RPPS'].isna()].index.tolist()
print(indices_nan, len(indices_nan))

[8, 23, 27, 73, 74, 94, 112, 116, 117, 118, 121, 122, 123, 124, 127, 130, 131, 132, 135, 136, 138, 139, 140, 141, 142, 143, 145, 148, 150, 151, 152, 153, 156, 157, 158, 160, 161, 163, 166, 167, 169, 170, 172, 173, 175, 176, 177, 178, 179, 180, 182, 183, 184, 186, 187, 188, 190, 191, 192, 194, 196, 197, 198, 201, 202, 204, 209, 213, 214, 215, 216, 219, 220, 221, 222, 223, 225, 226, 227, 229, 230, 231, 233, 234, 235, 236, 238, 239, 240, 241, 245, 246, 247, 249, 250, 252, 254, 256, 259, 260, 262, 263, 264, 265, 267, 268, 270, 271, 272, 276, 277, 278, 279, 280, 281, 282, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 312, 315, 316, 317, 318, 319, 320, 321, 323, 324, 325, 326, 327, 328, 329, 331, 335, 337, 339, 344, 346, 347, 348, 349, 350, 351, 352, 353, 354, 356, 358, 359, 360, 365, 367, 368, 371, 372, 373, 374, 376, 378, 379, 380, 381, 382, 383, 384, 387, 388, 390, 392, 393, 395, 396, 397, 398, 399, 400, 4

In [29]:
data['Profile Cards'].iloc[999]

'{\'card_0\': \'Tarifs et remboursement\\n\\n\\n\\nConventionné\\nCarte Vitale acceptée\\nTiers payant : Sécurité sociale\\nVoir les tarifs\\nMoyens de paiement\\nChèques, espèces et carte bancaire\\nExpertises et actes\\nBilan podologique\\nIonophorèse\\nK-Tape\\nKinesio Taping\\nOrthonyxie\\nOrthoplastie\\nOrthèse plantaire\\nPodologie\\nPédicurie\\nSemelles orthopédiques\\nVerrue\', \'card_1\': "Carte et informations d\'accès\\n9 Avenue Ampère, 77420 Champs-sur-Marne\\nMoyens de transport\\nBus - Ampere (lignes 213 et 312)\\nBus - Noisy-Champs Rer - Descartes (ligne 212)\\nInformations pratiques\\n1er étage avec ascenseur\\nEntrée accessible\\nJ\'autorise le traitement d\'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus\\nsur la collecte et le traitement des données par Google\\nAFFICHER LA CARTE", \'card_2\': "Présentation\\nDepuis janvier 2010, je vous reçois dans mon cabinet à Champs-sur-Marne à deux pas de

In [30]:
data_combined_sans_nan = data_combined[data_combined['RPPS'].notna()]
data_combined_sans_nan

,Doctor Name,Specialty,card_0,card_3,card_4,Latitude,Longitude,Skills Processed,RPPS,Nb Skills,...,Parking,Infos,Coordonnées,Sans RDV,nb terms card_0,nb terms card_3,nb terms card_4,nb carac card_0,nb carac card_3,nb carac card_4
0,Dr Marian AGACHI,Neurochirurgien,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.878328,2.431034,"[Expertises et actes, Neurochirurgie du rachis]",10001649291,1,...,0,1,1,0,0,8,0,0,64,0
1,Dr Anne Vaillant Moga,Médecin généraliste,"[Lieu 1, Lieu 2, Dr Anne Moga (Paris), 76 aven...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation préalable de médecine es...",48.869218,2.285468,"[Expertises et actes, Acné, Allergie, Apnée du...",10000378991,10,...,0,1,0,0,25,6,32,625,36,1024
2,Dr Gabriel Ohana,Chirurgien-dentiste,"[Tarifs et remboursement, , Carte Vitale accep...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation dentaire, 23 €, Traiteme...",48.894148,2.250698,"[Expertises et actes, Chirurgie buccale, Proth...",10003639373,3,...,0,1,1,0,12,8,10,144,64,100
3,Dr Thomas BAMBERGER,Dermatologue et vénérologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation, 31,50 € à 56,50 €, Cont...",48.815433,2.343017,NaN,10003719571,0,...,0,1,1,0,8,8,10,64,64,100
4,Dr Layla Chatila,Chirurgien-dentiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Coordonnées, 01 45 2...","[Tarifs, Consultation dentaire, 23 €, Ces hono...",48.858804,2.274392,NaN,10005133003,0,...,1,1,1,0,9,3,4,81,9,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,Dr Danielle Taconet,Ophtalmologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Contact d'urgence, E...",NaN,49.060704,2.184674,"[Expertises et actes, Champ visuel, Laser opht...",10003749230,4,...,0,1,1,0,13,5,0,169,25,0
994,Dr Stella Danan-Hadida,Gynécologue médicale et obstétrique,"[Tarifs et remboursement, , , , Conventionné s...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation vidéo, 58 €, Consultatio...",48.871909,2.346805,"[Expertises et actes, Gynécologie obstétrique,...",10000292291,7,...,1,1,1,0,18,8,28,324,64,784
995,Dr Philippe Lazare SELLAM,Spécialiste en chirurgie plastique reconstruct...,"[Lieu 1, Lieu 2, Cabinet du Docteur Philippe L...","[Horaires et coordonnées, Contact d'urgence, E...","[Tarifs, Consultation, 23 € à 120 €, Exérèse d...",48.868676,2.325944,NaN,10000558964,0,...,1,1,1,0,11,5,8,121,25,64
996,Dr Michèle Maizil,Chirurgien-dentiste,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.834970,2.246637,NaN,10000202985,0,...,1,1,1,0,0,8,0,0,64,0


In [31]:
data_combined_sans_nan['RPPS'].head(10)

0     10001649291
1     10000378991
2     10003639373
3     10003719571
4     10005133003
5     10001538924
6     10006076227
7     10100273183
9     10004057633
10    10001025484
Name: RPPS, dtype: object

# Ouverture base de données activités

In [32]:
annuaire = pd.read_csv('data/PS_LibreAcces_Personne_activite_small.csv')
annuaire.shape

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_2810/2401431438.py:1: DtypeWarning: Columns (10,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  annuaire = pd.read_csv('data/PS_LibreAcces_Personne_activite_small.csv')


(83743, 27)

In [33]:
annuaire.isnull().sum()

identifiantpp                           0
identificationnationalepp               0
codecivilité                            0
nomdexercice                            1
prénomdexercice                         1
codecatégorieprofessionnelle            0
codesavoirfaire                         0
libellésavoirfaire                      0
codemodeexercice                        0
numérosiretsite                     58015
numérofinesssite                    57014
identifianttechniquedelastructur        0
raisonsocialesite                       0
numérovoiecoordstructure            10187
indicerépétitionvoiecoordstructu    78376
codetypedevoiecoordstructure        57627
libellétypedevoiecoordstructure     57627
libellévoiecoordstructure            1195
bureaucedexcoordstructure               0
codepostalcoordstructure                0
codecommunecoordstructure               1
libellécommunecoordstructure            1
libellépayscoordstructure           26729
codesecteurdactivité              

In [34]:
annuaire['identifiantpp']=annuaire['identifiantpp'].astype(str)

# Merge doctolib généraliste et base de données

In [35]:
medecin_gen = data_combined_sans_nan[data_combined_sans_nan['Specialty']=='Médecin généraliste']

In [36]:
medecin_gen['RPPS'].head()

1     10000378991
10    10001025484
16    10002991841
18    10001627800
19    10100469658
Name: RPPS, dtype: object

In [37]:
liste_med = annuaire.merge(medecin_gen,how='outer',left_on='identifiantpp',right_on='RPPS')

In [38]:
liste_med['identifiantpp'].head() 

0    10000003870
1    10000003870
2    10000004688
3    10000005180
4    10000005537
Name: identifiantpp, dtype: object

In [39]:
def departement(codepostal):
    code = int(codepostal)/1000
    if int(code) == 97 :
        return(str(int(codepostal/100)))
    if int(code) < 10 :
        return('0'+str(int(code)))
    return (str(int(code)))

In [40]:
liste_med.loc[:, 'code dépar'] = liste_med['codepostalcoordstructure'].apply(departement)

Ouverture concurrence d'après INSEE

In [41]:
concu = pd.read_excel('data/TCRD_068.xlsx')
concu.rename(columns={'Professionnels de santé au 1ᵉʳ janvier 2023 : comparaisons départementales' : 'Code Département','Unnamed: 1' : 'Nom Département','Unnamed: 2':'Nombre de médecins','Unnamed: 3':'Nombre de médecins pour 100 000 habitants','Unnamed: 4':'généralistes','Unnamed: 5':'spécialistes','Unnamed: 6':'dentistes','Unnamed: 7':'Pharm.'}, inplace=True)
concu.drop(labels = [0,1,2,3,108],axis=0, inplace=True)
concu

,Code Département,Nom Département,Nombre de médecins,Nombre de médecins pour 100 000 habitants,généralistes,spécialistes,dentistes,Pharm.
4,01,Ain,1162,174,99,75,53,78
5,02,Aisne,1107,211,99,111,45,91
6,03,Allier,917,275,134,141,51,111
7,04,Alpes-de-Haute-Provence,483,291,165,125,54,103
8,05,Hautes-Alpes,705,503,291,213,72,135
...,...,...,...,...,...,...,...,...
103,972,Martinique,1171,332,152,180,68,99
104,973,Guyane,717,242,123,120,30,46
105,974,La Réunion,3170,364,176,189,67,94
106,976,Mayotte,265,89,49,39,9,31


In [42]:
liste_med = liste_med.merge(concu[['Code Département','généralistes']], left_on='code dépar', right_on='Code Département',how='inner')

In [43]:
liste_med['RPPS'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: RPPS, dtype: object

In [44]:
data_final = liste_med[['nomdexercice', 'prénomdexercice','RPPS','Specialty','card_0', 'card_3', 'card_4','Skills Processed','Nb Skills', 'moyens_paiement',
       'carte_vitale', 'secteur', 'Présentation', 'Langues parlées',
       'Diplômes nationaux et universitaires', 'Autres formations',
       'Expériences', 'Travaux et publications',
       'Prix et distinctions', 'site', 'Formations',
       'Diplômes nationaux et universitaires b', 'Nb caractères présentation',
       'Autres formations b', 'Nb langues', 'Expériences b', 'Carac_card_1',
       'Transport', 'Parking', 'Infos', 'Coordonnées', 'Sans RDV','Code Département',
       'généralistes']]
data_final.head()

,nomdexercice,prénomdexercice,RPPS,Specialty,card_0,card_3,card_4,Skills Processed,Nb Skills,moyens_paiement,...,Nb langues,Expériences b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV,Code Département,généralistes
0,VUILLET,MICHEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,150
1,VUILLET,MICHEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,150
2,WEILL,EDGARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,150
3,BORDESSOULES,PIERRE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,150
4,DORVILLE,ALBERT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,971,150


# Exporter

In [45]:
liste_med.to_csv('liste_med.csv')
medecin_gen.to_csv('medecin_gen.csv')

In [46]:
liste_med_ban=liste_med[liste_med['numérovoiecoordstructure'].notnull()]

In [47]:
def nom_voie(nom_voie):
    if isinstance(nom_voie,str):
        if nom_voie[0] in ['1','2','3','4','5','6','7','8','9']:
            nom_voie = nom_voie[1:]
        if 'Imp ' in nom_voie:
            return nom_voie.replace('Imp ','Impasse ').lower()
        else:
            return nom_voie.lower()
    else:
        return nom_voie.lower()

In [48]:
liste_med_ban=liste_med_ban[liste_med_ban['libellépayscoordstructure']!='Belgique']

In [49]:
def fct_voie(type_voie,nom_voie):
    result = ''
    if isinstance(type_voie,str) and isinstance(nom_voie,str):
        result = (type_voie+' '+nom_voie).lower()
    elif isinstance(nom_voie,str):
        result = nom_voie.lower()
    else:
        result = nom_voie
    if isinstance(result,str) and result[0]=='-':
        if result[1]==' ':
            result= result[2:]
        else :
            result = result[1:]
    if isinstance(result,str) and result[0]=='/':
        result = result[1:]
    return result

In [50]:
liste_med_ban['Voie'] = liste_med_ban.apply(lambda x: fct_voie(x['libellétypedevoiecoordstructure'],x['libellévoiecoordstructure']),axis=1)

In [51]:
liste_med_ban['RPPS'].head()

1     NaN
7     NaN
10    NaN
14    NaN
15    NaN
Name: RPPS, dtype: object

# Ouverture base de données centroides

In [52]:
import geopandas as gpd

In [53]:
gdf = gpd.read_file('data/a-com2022-topo-2154.json')
gdf.head()

,id,codgeo,dep,reg,xcl2154,ycl2154,libgeo,geometry
0,None,01001,01,84,848241,6563021,L'Abergement-Clémenciat,"POLYGON ((846981.436 6564107.360, 847017.457 6..."
1,None,01002,01,84,887495,6548152,L'Abergement-de-Varey,"POLYGON ((886027.651 6548170.900, 886157.325 6..."
2,None,01004,01,84,882724,6542583,Ambérieu-en-Bugey,"POLYGON ((884889.404 6539629.931, 884795.750 6..."
3,None,01005,01,84,847277,6545791,Ambérieux-en-Dombes,"POLYGON ((845951.250 6547999.078, 846073.720 6..."
4,None,01006,01,84,902191,6519791,Ambléon,"POLYGON ((901336.361 6521123.305, 902460.201 6..."


In [54]:
def codecom2(codecom1):
    if codecom1.startswith('75'):
        return('75056')
    elif codecom1.startswith('693'):
        return('69123')
    else :
        return(codecom1)

In [55]:
liste_med_ban['codecommunecoordstructure2']=liste_med_ban['codecommunecoordstructure'].apply(codecom2)

In [56]:
liste_med_ban_metr = liste_med_ban[liste_med_ban['Code Département']!='971']

In [57]:
liste_med_ban['RPPS'].head(10)

1     NaN
7     NaN
10    NaN
14    NaN
15    NaN
17    NaN
20    NaN
21    NaN
23    NaN
25    NaN
Name: RPPS, dtype: object

In [58]:
df_merged_gdf = liste_med_ban_metr.merge(gdf, how='inner', left_on='codecommunecoordstructure2', right_on='codgeo')

In [59]:
print(liste_med_ban_metr.shape,liste_med_ban.shape,df_merged_gdf.shape)

(73252, 74) (73374, 74) (72481, 82)


In [60]:
df_merged_gdf.head(1415)

,identifiantpp,identificationnationalepp,codecivilité,nomdexercice,prénomdexercice,codecatégorieprofessionnelle,codesavoirfaire,libellésavoirfaire,codemodeexercice,numérosiretsite,...,Voie,codecommunecoordstructure2,id,codgeo,dep,reg,xcl2154,ycl2154,libgeo,geometry
0,10000010867,810000010867,M,TACONET,CHRISTIAN,C,SM53,Spécialiste en Médecine Générale,L,NaN,...,avenue martin luther king,95637,None,95637,95,11,628442,6881566,Vauréal,"POLYGON ((627659.853 6880531.288, 627609.424 6..."
1,10000011808,810000011808,MME,HAMOT,ENNA,C,SM26,Qualifié en Médecine Générale,S,7.500959e+13,...,quai de jemmapes,75056,None,75056,75,11,652492,6862009,Paris,"POLYGON ((653493.751 6867050.446, 654876.938 6..."
2,10000012129,810000012129,M,DIDDEN,JEAN-PAUL,C,SM53,Spécialiste en Médecine Générale,L,NaN,...,avenue de la jonquiere,11304,None,11304,11,76,633310,6197645,Quillan,"POLYGON ((637068.406 6195234.866, 636830.671 6..."
3,10000012558,810000012558,M,CHUBERRE,YVES,C,SM53,Spécialiste en Médecine Générale,L,NaN,...,place de la chapelle,56034,None,56034,56,53,243618,6738059,Carnac,"POLYGON ((241700.262 6738105.440, 241988.426 6..."
4,10000013531,810000013531,M,COSTA,Frédéric,C,SM26,Qualifié en Médecine Générale,L,NaN,...,rue de rosny,93048,None,93048,93,11,658989,6862633,Montreuil,"POLYGON ((657160.638 6861172.713, 657095.801 6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410,10000634583,810000634583,MME,BENYAHIA,BAYA,C,SM53,Spécialiste en Médecine Générale,L,NaN,...,avenue de la republique,78089,None,78089,78,11,596081,6882855,Bonnières-sur-Seine,"POLYGON ((594837.979 6882621.785, 595774.512 6..."
1411,10000634955,810000634955,MME,BERREBI,DOMINIQUE,C,SM26,Qualifié en Médecine Générale,S,9.278086e+13,...,avenue cesar caire,75056,None,75056,75,11,652492,6862009,Paris,"POLYGON ((653493.751 6867050.446, 654876.938 6..."
1412,10000636810,810000636810,M,DIALLO,DIOUGA,C,SM54,Médecine Générale,L,NaN,...,allee de la scierie,35243,None,35243,35,53,383321,6813264,Romagné,"POLYGON ((385047.419 6810385.090, 383995.621 6..."
1413,10000636992,810000636992,M,AL KAHEF,HASSAN,C,SM54,Médecine Générale,S,2.000635e+13,...,rue victor hugo,77305,None,77305,77,11,696755,6809401,Montereau-Fault-Yonne,"POLYGON ((698886.778 6808752.784, 697345.101 6..."


In [61]:
liste_med_ban_metr.iloc[1414]

identifiantpp                             10000639137
identificationnationalepp                810000639137
codecivilité                                        M
nomdexercice                                    LEROY
prénomdexercice                                MICHEL
                                       ...           
code dépar                                         77
Code Département                                   77
généralistes                                       89
Voie                          avenue de fontainebleau
codecommunecoordstructure2                      77491
Name: 1617, Length: 74, dtype: object

In [62]:
gdf[gdf['codgeo'].str.startswith('7749')]

,id,codgeo,dep,reg,xcl2154,ycl2154,libgeo,geometry
30434,None,77490,77,11,706772,6882939,Vendrest,"POLYGON ((711011.276 6880853.454, 710233.233 6..."
30435,None,77492,77,11,727001,6863919,Verdelot,"POLYGON ((724201.981 6867687.618, 725066.473 6..."
30436,None,77493,77,11,687943,6838347,Verneuil-l'Étang,"POLYGON ((688448.039 6835707.309, 687569.139 6..."
30437,None,77494,77,11,688668,6809877,Vernou-la-Celle-sur-Seine,"POLYGON ((686041.870 6810163.154, 685832.951 6..."
30438,None,77495,77,11,671275,6829443,Vert-Saint-Denis,"POLYGON ((671309.488 6826343.028, 670992.507 6..."
30439,None,77496,77,11,710186,6830934,Vieux-Champagne,"POLYGON ((709203.047 6830617.092, 708180.065 6..."
30440,None,77498,77,11,686035,6869982,Vignely,"POLYGON ((687554.730 6870429.605, 687086.464 6..."


In [63]:
def nombre_de_med(codeCom):
    data_f = liste_med_ban_metr[(liste_med_ban_metr['codecommunecoordstructure']==codeCom)]
    return(data_f.shape[0])

liste_med_ban_metr['nb medecin commune']=liste_med_ban_metr['codecommunecoordstructure'].apply(nombre_de_med)

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_2810/981943077.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liste_med_ban_metr['nb medecin commune']=liste_med_ban_metr['codecommunecoordstructure'].apply(nombre_de_med)


In [64]:
liste_med_ban_metr.shape

(73252, 75)

In [65]:
df_merged_gdf = df_merged_gdf[['identifiantpp', 'codecivilité',
       'nomdexercice', 'prénomdexercice', 'codecatégorieprofessionnelle',
       'codesavoirfaire', 'libellésavoirfaire', 'codemodeexercice',
       'numérosiretsite',
       'identifianttechniquedelastructur', 'raisonsocialesite',
       'numérovoiecoordstructure', 'indicerépétitionvoiecoordstructu',
       'codetypedevoiecoordstructure', 'libellétypedevoiecoordstructure',
       'libellévoiecoordstructure', 'bureaucedexcoordstructure',
       'codepostalcoordstructure', 'codecommunecoordstructure',
       'libellécommunecoordstructure', 'libellépayscoordstructure',
       'codesecteurdactivité', 'libellésecteurdactivité', 'coderôle',
       'libellérôle', 'Doctor Name', 'Specialty', 'card_0', 'card_3', 'card_4',
       'Latitude', 'Longitude', 'Skills Processed', 'RPPS', 'Nb Skills',
       'moyens_paiement', 'carte_vitale', 'secteur', 'Présentation',
       'Langues parlées', 'Diplômes nationaux et universitaires',
       'Autres formations', 'Expériences', 'Travaux et publications',
       'Prix et distinctions', 'site', 'Site web', 'Formations',
       'Diplômes nationaux et universitaires b', 'Nb caractères présentation',
       'Autres formations b', 'Travaux et publications b', 'Nb langues',
       'Expériences b', 'Formations b', 'Carac_card_1', 'Transport', 'Parking',
       'Infos', 'Coordonnées', 'Sans RDV', 'nb terms card_0',
       'nb terms card_3', 'nb terms card_4', 'nb carac card_0',
       'nb carac card_3', 'nb carac card_4', 'code dépar', 'Code Département',
       'généralistes', 'Voie', 'codgeo', 'xcl2154',
       'ycl2154']]

In [66]:
df_merged_gdf.shape

(72481, 74)

### tentative jointure cross

In [67]:
# Jointure croisée avec 'how=cross'
df_cross_0 = pd.merge(df_merged_gdf[df_merged_gdf['identifiantpp']==df_merged_gdf['identifiantpp'].iloc[1]], df_merged_gdf, how='cross')
df_cross_0.head()

,identifiantpp_x,codecivilité_x,nomdexercice_x,prénomdexercice_x,codecatégorieprofessionnelle_x,codesavoirfaire_x,libellésavoirfaire_x,codemodeexercice_x,numérosiretsite_x,identifianttechniquedelastructur_x,...,nb carac card_0_y,nb carac card_3_y,nb carac card_4_y,code dépar_y,Code Département_y,généralistes_y,Voie_y,codgeo_y,xcl2154_y,ycl2154_y
0,10000011808,MME,HAMOT,ENNA,C,SM26,Qualifié en Médecine Générale,S,7.500959e+13,F750831182,...,NaN,NaN,NaN,95,95,95,avenue martin luther king,95637,628442,6881566
1,10000011808,MME,HAMOT,ENNA,C,SM26,Qualifié en Médecine Générale,S,7.500959e+13,F750831182,...,NaN,NaN,NaN,75,75,234,quai de jemmapes,75056,652492,6862009
2,10000011808,MME,HAMOT,ENNA,C,SM26,Qualifié en Médecine Générale,S,7.500959e+13,F750831182,...,NaN,NaN,NaN,11,11,143,avenue de la jonquiere,11304,633310,6197645
3,10000011808,MME,HAMOT,ENNA,C,SM26,Qualifié en Médecine Générale,S,7.500959e+13,F750831182,...,NaN,NaN,NaN,56,56,178,place de la chapelle,56034,243618,6738059
4,10000011808,MME,HAMOT,ENNA,C,SM26,Qualifié en Médecine Générale,S,7.500959e+13,F750831182,...,NaN,NaN,NaN,93,93,99,rue de rosny,93048,658989,6862633


In [68]:
df_cross_0.to_csv('df_cross_0.csv')

In [73]:
df_cross_0.columns[10:]

Index(['raisonsocialesite_x', 'numérovoiecoordstructure_x',
       'indicerépétitionvoiecoordstructu_x', 'codetypedevoiecoordstructure_x',
       'libellétypedevoiecoordstructure_x', 'libellévoiecoordstructure_x',
       'bureaucedexcoordstructure_x', 'codepostalcoordstructure_x',
       'codecommunecoordstructure_x', 'libellécommunecoordstructure_x',
       ...
       'nb carac card_0_y', 'nb carac card_3_y', 'nb carac card_4_y',
       'code dépar_y', 'Code Département_y', 'généralistes_y', 'Voie_y',
       'codgeo_y', 'xcl2154_y', 'ycl2154_y'],
      dtype='object', length=138)

In [179]:
df_merged_gdf.head()

,identifiantpp,codecivilité,nomdexercice,prénomdexercice,codecatégorieprofessionnelle,codesavoirfaire,libellésavoirfaire,codemodeexercice,numérosiretsite,identifianttechniquedelastructur,...,nb carac card_0,nb carac card_3,nb carac card_4,code dépar,Code Département,généralistes,Voie,codgeo,xcl2154,ycl2154
0,10000010867,M,TACONET,CHRISTIAN,C,SM53,Spécialiste en Médecine Générale,L,NaN,R10000000618793,...,NaN,NaN,NaN,95,95,95,avenue martin luther king,95637,628442,6881566
1,10000011808,MME,HAMOT,ENNA,C,SM26,Qualifié en Médecine Générale,S,7.500959e+13,F750831182,...,NaN,NaN,NaN,75,75,234,quai de jemmapes,75056,652492,6862009
2,10000012129,M,DIDDEN,JEAN-PAUL,C,SM53,Spécialiste en Médecine Générale,L,NaN,R10100000269875,...,NaN,NaN,NaN,11,11,143,avenue de la jonquiere,11304,633310,6197645
3,10000012558,M,CHUBERRE,YVES,C,SM53,Spécialiste en Médecine Générale,L,NaN,R10000000562949,...,NaN,NaN,NaN,56,56,178,place de la chapelle,56034,243618,6738059
4,10000013531,M,COSTA,Frédéric,C,SM26,Qualifié en Médecine Générale,L,NaN,R10100000568503,...,NaN,NaN,NaN,93,93,99,rue de rosny,93048,658989,6862633


In [73]:
import pandas as pd

# Exemple de DataFrames
df1 = pd.DataFrame({'A': [1, 2, 3]})
df2 = pd.DataFrame({'B': ['a', 'b']})

# Jointure croisée avec 'how=cross'
df_cross = pd.merge(df1[df1['A']==df1['A'].iloc[0]], df1, how='cross')

print(df_cross)

   A_x  A_y
0    1    1
1    1    2
2    1    3
